In [ ]:
import os
import sys
from pathlib import Path

import datarobot as dr
from dotenv import load_dotenv

# The notebook should be executed from the project root directory
if "_correct_path" not in locals():
    os.chdir("..")
    sys.path.append(".")
    print(f"changed dir to {Path('.').resolve()})")
    _correct_path = True
load_dotenv()
client = dr.Client()

In [ ]:
from datarobotx.idp.use_cases import get_or_create_use_case

from infra.settings_main import use_case_args

if "DATAROBOT_DEFAULT_USE_CASE" in os.environ:
    use_case_id = os.environ["DATAROBOT_DEFAULT_USE_CASE"]
else:
    use_case_id = get_or_create_use_case(
        endpoint=client.endpoint,
        token=client.token,
        name=use_case_args.resource_name,
        description=use_case_args.description,
    )

# Scoring Data Ingest and Preparation

In [ ]:
import pandas as pd

from infra.settings_datasets import scoring_dataset

# Replace as needed with your own data ingest and/or preparation logic
df = pd.read_csv(scoring_dataset.file_path)

In [ ]:
from datarobotx.idp.datasets import get_or_create_dataset_from_df

print("Uploading scoring dataset...")
scoring_dataset_id = get_or_create_dataset_from_df(
    endpoint=client.endpoint,
    token=client.token,
    data_frame=df,
    name=scoring_dataset.resource_name,
    use_cases=use_case_id,
)

# Export dataset id for provisioning app, other dependent resources

In [ ]:
import yaml

from forecastic.resources import ScoringDataset
from infra.settings_main import scoring_prep_output_file

with open(scoring_prep_output_file, "w") as f:
    yaml.dump(ScoringDataset(id=scoring_dataset_id).model_dump(), f, allow_unicode=True)